In [2]:
from sklearn.metrics.pairwise import cosine_similarity
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd 
import numpy as np 
from sklearn.feature_extraction.text import TfidfVectorizer

In [4]:
cast=pd.read_csv('/Users/apple/Desktop/40000/credits.csv')
df2=pd.read_csv('/Users/apple/Desktop/40000/movies_metadata.csv')
cast['id']=cast['id'].astype(str)
df2['id']=df2['id'].astype(str)

/Users/apple/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [64]:
#join the data on id
new = pd.merge(cast,df2,on = "id",how = "left").drop_duplicates()

,cast,crew,id,adult,belongs_to_collection,budget,genres,homepage,imdb_id,original_language,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,tt0114709,en,...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",8844,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,tt0113497,en,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...",15602,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,tt0113228,en,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,"[{'cast_id': 1, 'character': ""Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de...",31357,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,tt0114885,en,...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de...",11862,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,tt0113041,en,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45533,"[{'cast_id': 0, 'character': '', 'credit_id': ...","[{'credit_id': '5894a97d925141426c00818c', 'de...",439050,False,NaN,0,"[{'id': 18, 'name': 'Drama'}, {'id': 10751, 'n...",http://www.imdb.com/title/tt6209470/,tt6209470,fa,...,NaN,0.0,90.0,"[{'iso_639_1': 'fa', 'name': 'فارسی'}]",Released,Rising and falling between a man and woman,Subdue,False,4.0,1.0
45534,"[{'cast_id': 1002, 'character': 'Sister Angela...","[{'credit_id': '52fe4af1c3a36847f81e9b15', 'de...",111109,False,NaN,0,"[{'id': 18, 'name': 'Drama'}]",NaN,tt2028550,tl,...,2011-11-17,0.0,360.0,"[{'iso_639_1': 'tl', 'name': ''}]",Released,NaN,Century of Birthing,False,9.0,3.0
45535,"[{'cast_id': 6, 'character': 'Emily Shaw', 'cr...","[{'credit_id': '52fe4776c3a368484e0c8387', 'de...",67758,False,NaN,0,"[{'id': 28, 'name': 'Action'}, {'id': 18, 'nam...",NaN,tt0303758,en,...,2003-08-01,0.0,90.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,A deadly game of wits.,Betrayal,False,3.8,6.0
45536,"[{'cast_id': 2, 'character': '', 'credit_id': ...","[{'credit_id': '533bccebc3a36844cf0011a7', 'de...",227506,False,NaN,0,[],NaN,tt0008536,en,...,1917-10-21,0.0,87.0,[],Released,NaN,Satan Triumphant,False,0.0,0.0


# Demographic Filtering based on weighted ratings

In [6]:
## Trending Recemmendations

#must have more votes than at least 60% of the movies in the list.
#50% is appropriate since the minimum # of voters is 15 when set to 0.6
C= new['vote_average'].mean()
m= new['vote_count'].quantile(0.60)
q_movies =new.copy().loc[new['vote_count'] >= m] #only movies with >=m counts of raters should be considered

def weighted_rating(x, m=m, C=C):
    v = x['vote_count']
    R = x['vote_average']
    # Calculation based on the IMDB formula
    return (v/(v+m) * R) + (m/(m+v) * C)

# Define a new feature 'score' and calculate its value with `weighted_rating()`
q_movies['score'] = q_movies.apply(weighted_rating, axis=1)
q_movies = q_movies.sort_values('score', ascending=False) #sort based on score
q_movies
q_movies[['title', 'vote_count', 'vote_average', 'score']].head(15) #print top 15 movies

,title,vote_count,vote_average,score
10325,Dilwale Dulhania Le Jayenge,661.0,9.1,9.022742
43389,Planet Earth II,50.0,9.5,8.604207
39153,Planet Earth,176.0,8.8,8.550123
314,The Shawshank Redemption,8358.0,8.5,8.494837
835,The Godfather,6024.0,8.5,8.492842
40320,Your Name.,1030.0,8.5,8.458635
12501,The Dark Knight,12269.0,8.3,8.296725
2848,Fight Club,9678.0,8.3,8.295850
292,Pulp Fiction,8670.0,8.3,8.295368
522,Schindler's List,4436.0,8.3,8.290962


# Content Based Filtering: recommend similar movies

### 1. Consider only overview, later may want to consider the ratings into the recommendations

In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer
#Define a TF-IDF Vectorizer Object. Remove all english stop words such as 'the', 'a'
tfidf = TfidfVectorizer(stop_words='english')

#Replace NaN with an empty string
new['overview'] = new['overview'].fillna('')

#Construct the required TF-IDF matrix by fitting and transforming the data
tfidf_matrix = tfidf.fit_transform(new['overview'])

#Output the shape of tfidf_matrix
tfidf_matrix.shape #over 75827 words to describe 45538 movies

(45538, 75827)

In [ ]:
# Import linear_kernel
from sklearn.metrics.pairwise import linear_kernel
# Compute the cosine similarity matrix
#下面一行跑完会崩，需要用gpu
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)
#Construct a reverse map to identify the index of a movie in our metadata DataFrame, given its title
indices = pd.Series(new.index, index=new['title']).drop_duplicates()
def get_recommendations(title, cosine_sim=cosine_sim):
    # Get the index of the movie that matches the title
    idx = indices[title]

    # Get the pairwsie similarity scores of all movies with that movie
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Sort the movies based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get the scores of the 10 most similar movies
    sim_scores = sim_scores[1:11]

    # Get the movie indices
    movie_indices = [i[0] for i in sim_scores]

    # Return the top 10 most similar movies
    return new['title'].iloc[movie_indices]

In [ ]:
get_recommendations('The Dark Knight Rises') 
# gives all batman related films when users might want films of the same genres or directors

### 2. Consider: credits, genres, keywords, cast, directors

In [67]:
#join keywords on id
keywords = pd.read_csv('/Users/apple/Desktop/40000/keywords.csv')
keywords['id']=keywords['id'].astype(str)
#keywords.head()
#New = pd.merge(new,keywords,on = "id",how = "left").drop_duplicates()
New = new.reset_index().merge(keywords, how="left").set_index('index').drop_duplicates()

In [70]:
# Parse the stringified features into their corresponding python objects
from ast import literal_eval
features = ['cast', 'crew', 'keywords', 'genres']
for feature in features:
    New[feature] = New[feature].apply(literal_eval)

In [71]:
#get directors, if no directors return na
def get_director(x):
    for i in x:
        if i['job'] == 'Director':
            return i['name']
    return np.nan

# Returns the list top 3 elements or entire list; whichever is more.
def get_list(x):
    if isinstance(x, list):
        names = [i['name'] for i in x]
        #Check if more than 3 elements exist. If yes, return only first three. If no, return entire list.
        if len(names) > 3:
            names = names[:3]
        return names

    #Return empty list in case of missing/malformed data
    return []

In [72]:
# Define new director, cast, genres and keywords features that are in a suitable form.
New['director'] = New['crew'].apply(get_director)

# extract the first three items for each feature
features = ['cast', 'keywords', 'genres']
for feature in features:
    New[feature] = New[feature].apply(get_list)
    
New[['title', 'cast', 'director', 'keywords', 'genres']].head(3)

,title,cast,director,keywords,genres
index,,,,,
0,Toy Story,"[Tom Hanks, Tim Allen, Don Rickles]",John Lasseter,"[jealousy, toy, boy]","[Animation, Comedy, Family]"
1,Jumanji,"[Robin Williams, Jonathan Hyde, Kirsten Dunst]",Joe Johnston,"[board game, disappearance, based on children'...","[Adventure, Fantasy, Family]"
2,Grumpier Old Men,"[Walter Matthau, Jack Lemmon, Ann-Margret]",Howard Deutch,"[fishing, best friend, duringcreditsstinger]","[Romance, Comedy]"


In [73]:
# Function to convert all strings to lower case and strip names of spaces
def clean_data(x):
    if isinstance(x, list):
        return [str.lower(i.replace(" ", "")) for i in x]
    else:
        #Check if director exists. If not, return empty string
        if isinstance(x, str):
            return str.lower(x.replace(" ", ""))
        else:
            return ''
features = ['cast', 'keywords', 'director', 'genres']

for feature in features:
    New[feature] = New[feature].apply(clean_data)

In [74]:
New[['title', 'cast', 'director', 'keywords', 'genres']].head(3)

,title,cast,director,keywords,genres
index,,,,,
0,Toy Story,"[tomhanks, timallen, donrickles]",johnlasseter,"[jealousy, toy, boy]","[animation, comedy, family]"
1,Jumanji,"[robinwilliams, jonathanhyde, kirstendunst]",joejohnston,"[boardgame, disappearance, basedonchildren'sbook]","[adventure, fantasy, family]"
2,Grumpier Old Men,"[waltermatthau, jacklemmon, ann-margret]",howarddeutch,"[fishing, bestfriend, duringcreditsstinger]","[romance, comedy]"


In [75]:
## Final step before modeling
## Want to combine all the features into one feature
def create_soup(x):
    return ' '.join(x['keywords']) + ' ' + ' '.join(x['cast']) + ' ' +x['director']+' '+x['director']+ ' ' + ' '.join(x['genres'])
New['combine'] = New.apply(create_soup, axis=1)


In [93]:
NewH =New.head(30)
#New.to_csv("/Users/apple/Desktop/40000/New.csv",index=False)

In [80]:
# Import CountVectorizer and create the count matrix
from sklearn.feature_extraction.text import CountVectorizer
count = CountVectorizer(stop_words='english')
count_matrix = count.fit_transform(NewH['combine'])

# Compute the Cosine Similarity matrix based on the count_matrix
from sklearn.metrics.pairwise import cosine_similarity
#下面一行跑完会崩，需要用gpu
cosine_sim2 = cosine_similarity(count_matrix, count_matrix) 

In [92]:
# Reset index of our main DataFrame and construct reverse mapping as before
indices = pd.Series(NewH.index, index=NewH['title'])
def get_recommendations(title, cosine_sim):
    # Get the index of the movie that matches the title
    idx = indices[title]

    # Get the pairwsie similarity scores of all movies with that movie
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Sort the movies based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get the scores of the 10 most similar movies
    sim_scores = sim_scores[1:11]

    # Get the movie indices
    movie_indices = [i[0] for i in sim_scores]

    # Return the top 10 most similar movies
    return New['title'].iloc[movie_indices]
get_recommendations('Toy Story', cosine_sim2)

12                             Balto
26                      Now and Then
4        Father of the Bride Part II
25                           Othello
6                            Sabrina
2                   Grumpier Old Men
3                  Waiting to Exhale
17                        Four Rooms
18    Ace Ventura: When Nature Calls
19                       Money Train
Name: title, dtype: object

# Recommendation System Design

In [ ]:
# Recommend 5 most similar films, given one movie
# 1. combine recommendations from two content-based to a dataframe
# 2. col1 = title, col2: counts, col3:popularity, col4:modified scores, col5 = orders

# Collaborative Filtering